In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import os

# Define dataset paths
train_dir = "D:/Major Project/normalized/train"
valid_dir = "D:/Major Project/normalized/valid"

# Image size and batch size optimized for RTX 3050 Ti
IMG_SIZE = (224, 224)  # Input image size
BATCH_SIZE = 32  # Reduce to 16 if memory issues occur

# Data augmentation for training set to improve generalization
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescale validation images, no augmentation
valid_datagen = ImageDataGenerator(rescale=1./255)

# Ensure only images are processed and dataset is correctly structured
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True  # Randomize order for better training
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  # Keep validation order consistent
)

# Define Custom CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),  # First convolution layer
    MaxPooling2D(2,2),  # Reduce feature map size

    Conv2D(64, (3,3), activation='relu'),  # Second convolution layer
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),  # Third convolution layer
    MaxPooling2D(2,2),

    Flatten(),  # Flatten feature maps into a 1D vector
    Dense(512, activation='relu'),  # Fully connected layer
    Dropout(0.5),  # Dropout to prevent overfitting
    Dense(4, activation='softmax')  # Output layer for 4 classes
])

# Compile the model with Adam optimizer
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping to stop training if validation loss stops improving
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Check if images are correctly loaded (Debugging Step)
batch_x, batch_y = next(train_generator)
print(f"Batch Shape (Images): {batch_x.shape}")  # Should print (BATCH_SIZE, 224, 224, 3)
print(f"Batch Shape (Labels): {batch_y.shape}")  # Should print (BATCH_SIZE, 4)

# Display a sample image from the dataset
plt.imshow(batch_x[0])
plt.title("Sample Image from Dataset")
plt.show()

# Check if training and validation generators are loading data
train_batch = next(iter(train_generator))
valid_batch = next(iter(valid_generator))

print(f"Training Batch Shape: {train_batch[0].shape}")  # Should print (Batch_Size, 224, 224, 3)
print(f"Validation Batch Shape: {valid_batch[0].shape}")  # Should print (Batch_Size, 224, 224, 3)


# Train the model
EPOCHS = 25  # Can be increased if needed
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

# Save the trained model
model.save("custom_cnn_teeth_health.h5")

# Print Model Summary
model.summary()


Found 16000 images belonging to 4 classes.
Found 4121 images belonging to 4 classes.
Batch Shape (Images): (32, 224, 224, 3)
Batch Shape (Labels): (32, 4)


Training Batch Shape: (32, 224, 224, 3)
Validation Batch Shape: (32, 224, 224, 3)
Epoch 1/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 389s 772ms/step - accuracy: 0.2574 - loss: 1.3936 - val_accuracy: 0.3424 - val_loss: 1.4073
Epoch 2/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 356s 709ms/step - accuracy: 0.3678 - loss: 1.3328 - val_accuracy: 0.4193 - val_loss: 1.4504
Epoch 3/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 355s 708ms/step - accuracy: 0.3961 - loss: 1.2868 - val_accuracy: 0.4336 - val_loss: 1.5331
Epoch 4/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 359s 715ms/step - accuracy: 0.4208 - loss: 1.2495 - val_accuracy: 0.4067 - val_loss: 1.6228


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │    44,302,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 133,194,446 (508.10 MB)

 Trainable params: 44,398,148 (169.37 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 88,796,298 (338.73 MB)